In [1]:
import os

api_key = ""
os.environ["OPENAI_API_KEY"] = api_key

In [2]:
!pip install openai==0.27.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


In [3]:
ds_name = "ml_latest_small"  #"ml_latest_small" # "lfm360k" #"ml_latest_small" #"ml_latest_small" #"ml_latest_small"
# Replace underscore with a hyphen
ds_name_n = ds_name.replace('_', '-')

use_precomputed = "Yes"
data_aug_with_rationale = "no"
flag_usePrecomputedAttack = "No"
attack_direction = 'promote'
llm_model = "openai" #openai #llama3 #sentence_transformer"
attackName = 'original' #original # 'emotional' ,'neighbor' 'chain' , 'trigger'
NUM_USERS   = 120  # Evaluate on 10 users for demonstration

ratiosList = [0.1, 0.4]
neighborList = [5, 15]

ratio = ratiosList[0]
n = neighborList[1]

if attackName == 'emotional':
  attack_type = f"final_text_phase1_{int(ratio*100)}p_{attack_direction}_emotional"
elif attackName == 'neighbor':
  attack_type = f"final_text_phase1_{int(ratio*100)}p_{attack_direction}_neighbor{n}"
elif attackName == 'chain':
  attack_type = f"final_text_phase1_{int(ratio*100)}p_{attack_direction}_chain{n}"
elif attackName == 'trigger':
  attack_type = f"final_text_phase1_{int(ratio*100)}p_{attack_direction}_trigger{n}"
else:
  attack_type = ""

print(ds_name_n)
print(attack_type)



ml-latest-small



In [4]:
import requests
import pandas as pd
import ast
import openai

def download_csv(url, csv_file_name):
    """
    Download a CSV file from a URL and save it locally.

    Parameters:
    url (str): The URL to download the file from.
    csv_file_name (str): The local filename to save the downloaded file.

    Returns:
    bool: True if download succeeds, False otherwise.
    """
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(csv_file_name, "wb") as file:
            for chunk in response.iter_content(chunk_size=128):
                file.write(chunk)
        print(f"{csv_file_name} downloaded successfully.")
        return True
    print(f"Failed to download the file. Status code: {response.status_code}")
    return False

# Function to load a CSV file (compressed or uncompressed)
def load_csv_file(file_path):
    """
    Load a CSV file, handling both compressed (.csv.gz) and uncompressed (.csv) formats.

    Parameters:
    - file_path (str): Path to the CSV file.

    Returns:
    - pd.DataFrame: Loaded DataFrame.
    """
    if file_path.endswith('.csv.gz'):
        return pd.read_csv(file_path, compression='gzip')
    elif file_path.endswith('.csv'):
        return pd.read_csv(file_path)
    else:
        raise ValueError(f"Unsupported file format: {file_path}")

def load_data_ml():
    """
    Download and load datasets for ratings, items, and tags, merging tags into items.

    Returns:
    tuple: DataFrames for ratings, items, tags, and items with merged tags.
    """
    urls = {
        "ratings": "https://raw.githubusercontent.com/yasdel/RecSys_Datasets/e3a18487525d3ae8ec19b39959c1a881c383e1a8/ml_latest/df_ratings_ml-latest-small.csv",
        "items": "https://raw.githubusercontent.com/yasdel/RecSys_Datasets/e3a18487525d3ae8ec19b39959c1a881c383e1a8/ml_latest/df_item_ml-latest-small.csv",
        "tags": "https://raw.githubusercontent.com/yasdel/RecSys_Datasets/21484da12f01263fc0194d12c63d796628f84137/ml_latest/df_tags_ml-latest-small.csv"
    }
    files = {key: f"df_{key}_ml-latest-small.csv" for key in urls}

    if all(download_csv(urls[key], files[key]) for key in urls):
        df_ratings = pd.read_csv(files["ratings"], names=['userId', 'itemId', 'rating', 'timestamp'], header=0, low_memory=False)
        df_items = pd.read_csv(files["items"], low_memory=False)
        df_tags = pd.read_csv(files["tags"], low_memory=False)

        for df, col in [(df_ratings, 'userId'), (df_ratings, 'itemId'), (df_items, 'itemId'), (df_tags, 'itemId'), (df_tags, 'userId')]:
            df[col] = df[col].astype(int)

        tags_grouped = df_tags.groupby('itemId')['tag'].apply(list).reset_index()
        df_items_merged = pd.merge(df_items, tags_grouped, on='itemId', how='left')
        df_items_merged['tag'] = df_items_merged['tag'].apply(lambda x: x if isinstance(x, list) else [])

        print("Datasets loaded successfully.")
        return df_ratings, df_items_merged, df_tags

    print("Failed to download one or more datasets.")
    return None, None, None, None

def load_data_lastfm(n=10):
    """
    Loads the LFM-360K dataset from GitHub, downloads if needed.
    Then converts 'plays' into a rating [0..n] using the formula:
        rating(u,i) = n * F(playcount_u_i),
    where F(playcount_u_i) is the empirical CDF of that item’s
    playcount among the user’s listened items.

    Returns:
        df_ratings (DataFrame): columns [userId, itemId, rating]
        df_items   (DataFrame): columns [itemId, artist, tags]
    """
    url = "https://raw.githubusercontent.com/yasdel/RecSys_Datasets/main/lfm-360K/lastFM360k_trg_user_40000_trg_Artist_4000.csv"
    out_file = "lastFM360k_trg_user_40000_trg_Artist_4000.csv"

    success = download_csv(url, out_file)
    if not success:
        return None, None

    # Load
    df = pd.read_csv(out_file, low_memory=False, index_col = 0)
    display(df)
    # Rename columns so we can unify them with typical 'itemId', 'rating'
    df.rename(columns={'artistId': 'itemId', 'plays': 'playcount'}, inplace=True)

    # Construct a ratings DataFrame
    df_ratings = df[['userId', 'itemId', 'playcount']].copy()

    # === Compute the user-specific CDF-based rating ===
    # We group by user, compute the rank of each playcount among that user's items
    # so rating = n * ( rank / total_items_in_user_profile ).
    # We do rank(method='min') for tie handling, you can also do 'dense' if you prefer.

    df_ratings['rank_in_user'] = df_ratings.groupby('userId')['playcount'] \
                                           .rank(method='min', ascending=True)

    # Number of items each user has
    df_ratings['count_in_user'] = df_ratings.groupby('userId')['playcount'] \
                                            .transform('count')

    # Empirical CDF: fraction of items that have playcount <= current playcount
    # We'll do (rank_in_user / count_in_user).
    # Then multiply by n to get a [0..n] rating.
    df_ratings['rating'] = (df_ratings['rank_in_user'] / df_ratings['count_in_user']) * n

    # Now we only keep userId, itemId, rating
    df_ratings = df_ratings[['userId', 'itemId', 'rating']].copy()

    # Construct an items DataFrame
    df_items = df[['itemId', 'artist', 'tags']].drop_duplicates().copy()

    # Convert 'tags' from string to list if needed
    df_items['tags'] = df_items['tags'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x
    )

    print("Last.fm dataset loaded and playcounts converted to rating scale [0..{}].".format(n))
    return df_ratings, df_items

def print_statistics(data):
    """
    Print dataset statistics including total interactions, unique users, items, and interaction ratios.

    Parameters:
    data (pd.DataFrame): DataFrame with 'userId' and 'itemId' columns.
    """
    display(data)
    total_interactions = data.shape[0]
    unique_users = data['userId'].nunique()
    unique_items = data['itemId'].nunique()

    print("--------------------------")
    print("The Dataset Overview:")
    print(f" - Total Interactions: {total_interactions}")
    print(f" - |U|: {unique_users}")
    print(f" - |I|: {unique_items}")
    print(f" - |R|/|U|: {total_interactions / unique_users:.2f}")
    print(f" - |R|/|I|: {total_interactions / unique_items:.2f}")
    print(f" - |R|/(|U|*|I|): {total_interactions / (unique_users * unique_items):.10f}")
    print("--------------------------")

def apply_interaction_limits(data, min_interactions, max_interactions, user_col, item_col):
    """
    Filter users based on interaction limits.

    Parameters:
    data (pd.DataFrame): DataFrame with interactions.
    min_interactions (int): Minimum number of interactions per user.
    max_interactions (int): Maximum number of interactions per user.
    user_col (str): Column name for users.
    item_col (str): Column name for items.

    Returns:
    pd.DataFrame: Filtered DataFrame.
    """
    interaction_counts = data[user_col].value_counts()
    users_to_keep = interaction_counts[(interaction_counts >= min_interactions) & (interaction_counts <= max_interactions)].index
    return data[data[user_col].isin(users_to_keep)]

display("Original Dataset Statistics:")



# Example: Download and load datasets
#df_ratings, df_items, df_tags = load_datasets()
#print_statistics(df_ratings)


# Main logic:
if ds_name == "ml_latest_small":
    df_ratings, df_items, df_tags = load_data_ml()  # returns three DFs for ML
    display(df_ratings.head())
    display(df_items.head())
    print_statistics(df_ratings)
    # Load the file from the local directory
    if data_aug_with_rationale.lower() == "no":
      fileName = "enriched_descriptions_dataset_ml_latest_small_gpt-4o-mini_with_rationale_No.csv.gz"
    else:
      fileName = "enriched_descriptions_dataset_ml_latest_small_gpt-4o-mini_with_rationale_Yes.csv.gz"
    print(f'{fileName} loaded ... from dataset {ds_name}')
    df_items =  pd.read_csv("https://github.com/yasdel/GenRecSys-LLM-DataAugmentation/raw/main/Data/" + fileName, compression='gzip')
elif ds_name == "lfm360k":
    df_ratings, df_items = load_data_lastfm(n=10)       # returns two DFs for Last.fm
    if data_aug_with_rationale.lower() == "no":
      fileName = "enriched_desc_ds_lfm360k_gpt-4o-mini_with_rationale_No.csv.gz"
    else:
      fileName = "enriched_desc_ds_lfm360k_gpt-4o-mini_with_rationale_Yes.csv.gz"
    print(f'{fileName} loaded ... from dataset {ds_name}')
    df_items =  pd.read_csv("https://github.com/yasdel/GenRecSys-LLM-DataAugmentation/raw/main/Data/" + fileName, compression='gzip')
    display(df_ratings.head())
    display(df_items.head())
    print_statistics(df_ratings)
else:
    raise ValueError("Unknown dataset_flag specified.")


# Check if 'tag' exists, then rename to 'tags'
if 'tags' in df_items.columns:
    df_items.rename(columns={'tags': 'tag'}, inplace=True)
    display(df_items.head())

# Example: Apply interaction limits if datasets are loaded
if ds_name == "ml_latest_small" and df_ratings is not None:
    MIN_INTERACTIONS, MAX_INTERACTIONS = 20, 100
    df_ratings_filtered = apply_interaction_limits(df_ratings, MIN_INTERACTIONS, MAX_INTERACTIONS, 'userId', 'itemId')

    display("Filtered Dataset Statistics:")
    print_statistics(df_ratings_filtered)

    sampled_item_ids = df_ratings_filtered['itemId'].unique()
    df_items_sampled = df_items[df_items['itemId'].isin(sampled_item_ids)]

    display("Sampled items with tags:")
    display(df_items_sampled.head())


'Original Dataset Statistics:'

df_ratings_ml-latest-small.csv downloaded successfully.
df_items_ml-latest-small.csv downloaded successfully.
df_tags_ml-latest-small.csv downloaded successfully.
Datasets loaded successfully.


,userId,itemId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


,itemId,title,genres,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[pixar, pixar, fun]"
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[fantasy, magic board game, Robin Williams, game]"
2,3,Grumpier Old Men (1995),Comedy|Romance,"[moldy, old]"
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,[]
4,5,Father of the Bride Part II (1995),Comedy,"[pregnancy, remake]"


,userId,itemId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


...

'Filtered Dataset Statistics:'

,userId,itemId,rating,timestamp
232,2,318,3.0,1445714835
233,2,333,4.0,1445715029
234,2,1704,4.5,1445715228
235,2,3578,4.0,1445714885
236,2,6874,4.0,1445714952
...,...,...,...,...
99529,609,892,3.0,847221080
99530,609,1056,3.0,847221080
99531,609,1059,3.0,847221054
99532,609,1150,4.0,847221054


--------------------------
The Dataset Overview:
 - Total Interactions: 16823
 - |U|: 365
 - |I|: 3079
 - |R|/|U|: 46.09
 - |R|/|I|: 5.46
 - |R|/(|U|*|I|): 0.0149692793
--------------------------


'Sampled items with tags:'

,itemId,title,genres,tag,enriched_description
0,7,Sabrina (1995),Comedy|Romance,['remake'],**Sabrina (1995)** is a delightful romantic co...
1,20,Money Train (1995),Action|Comedy|Crime|Drama|Thriller,[],**Money Train (1995)** is a thrilling blend of...
2,36,Dead Man Walking (1995),Crime|Drama,"['death penalty', 'Nun']",**Dead Man Walking (1995)** is a poignant and ...
3,17,Sense and Sensibility (1995),Drama|Romance,['Jane Austen'],**Sense and Sensibility (1995)** is a beautifu...
4,3,Grumpier Old Men (1995),Comedy|Romance,"['moldy', 'old']",**Grumpier Old Men (1995)** is a delightful se...


In [5]:
import pandas as pd
import requests
from google.colab import files
from io import BytesIO
import gzip

if ds_name == "ml_latest_small" and attack_direction in ["promote"]:
  # GitHub raw URL of the dataset
  url = "https://github.com/yasdel/Poison-RAG-Plus/raw/main/AttackData/Augmentation_Attack_Data/augmented_items_ml_latest_small_promote_sample_size_10percent.csv.gz"
elif ds_name == "ml_latest_small" and attack_direction in ["demote"]:
  # GitHub raw URL of the dataset
  url = "https://github.com/yasdel/Poison-RAG-Plus/raw/main/AttackData/Augmentation_Attack_Data/augmented_items_ml_latest_small_demote_sample_size_10percent.csv.gz"


# Download the file
response = requests.get(url)
if response.status_code == 200:
    with gzip.open(BytesIO(response.content), 'rt', encoding='utf-8') as f:
        df_items = pd.read_csv(f)
else:
    print(f"Failed to download file. HTTP Status Code: {response.status_code}")

# Display the first few rows
display(df_items.head())

print(f'Attacked Item IDS')
attacked_item_ids_list = df_items.loc[
    df_items[f'phase1_{attack_direction}_applied'] == True,
    'itemId'
].unique()

print(attacked_item_ids_list)

,itemId,title,genres,tag,enriched_description,pop_class,sim_items_200,sim_items_200_class_based,phase1_10p_promote_emotional,phase1_10p_promote_neighbor5,...,final_text_phase1_10p_promote_neighbor15,final_text_phase1_10p_promote_chain15,final_text_phase1_10p_promote_trigger15,final_text_phase1_40p_promote_emotional,final_text_phase1_40p_promote_neighbor5,final_text_phase1_40p_promote_chain5,final_text_phase1_40p_promote_trigger5,final_text_phase1_40p_promote_neighbor15,final_text_phase1_40p_promote_chain15,final_text_phase1_40p_promote_trigger15
0,7,Sabrina (1995),Comedy|Romance,['remake'],**Sabrina (1995)** is a delightful romantic co...,0,"[915, 360, 6899, 345, 2155, 295, 68, 287, 1549...","{0: [186, 500, 555, 597, 1197, 1569, 1777, 229...",NaN,NaN,...,**Sabrina (1995)** is a delightful romantic co...,**Sabrina (1995)** is a delightful romantic co...,**Sabrina (1995)** is a delightful romantic co...,**Sabrina (1995)** is a delightful romantic co...,**Sabrina (1995)** is a delightful romantic co...,**Sabrina (1995)** is a delightful romantic co...,**Sabrina (1995)** is a delightful romantic co...,**Sabrina (1995)** is a delightful romantic co...,**Sabrina (1995)** is a delightful romantic co...,**Sabrina (1995)** is a delightful romantic co...
1,20,Money Train (1995),Action|Comedy|Crime|Drama|Thriller,[],**Money Train (1995)** is a thrilling blend of...,1,"[1604, 5454, 124859, 3537, 2391, 3384, 2344, 4...","{0: [1, 21, 172, 296, 333, 434, 778, 1073, 122...",NaN,NaN,...,**Money Train (1995)** is a thrilling blend of...,**Money Train (1995)** is a thrilling blend of...,**Money Train (1995)** is a thrilling blend of...,**Money Train (1995)** is a thrilling blend of...,**Money Train (1995)** is a thrilling blend of...,**Money Train (1995)** is a thrilling blend of...,**Money Train (1995)** is a thrilling blend of...,**Money Train (1995)** is a thrilling blend of...,**Money Train (1995)** is a thrilling blend of...,**Money Train (1995)** is a thrilling blend of...
2,36,Dead Man Walking (1995),Crime|Drama,"['death penalty', 'Nun']",**Dead Man Walking (1995)** is a poignant and ...,0,"[726, 7883, 81, 54787, 4160, 336, 1114, 3258, ...","{0: [151, 253, 282, 288, 377, 587, 1246, 1923,...",NaN,NaN,...,**Dead Man Walking (1995)** is a poignant and ...,**Dead Man Walking (1995)** is a poignant and ...,**Dead Man Walking (1995)** is a poignant and ...,**Dead Man Walking (1995)** is a poignant and ...,**Dead Man Walking (1995)** is a poignant and ...,**Dead Man Walking (1995)** is a poignant and ...,**Dead Man Walking (1995)** is a poignant and ...,**Dead Man Walking (1995)** is a poignant and ...,**Dead Man Walking (1995)** is a poignant and ...,**Dead Man Walking (1995)** is a poignant and ...
3,17,Sense and Sensibility (1995),Drama|Romance,['Jane Austen'],**Sense and Sensibility (1995)** is a beautifu...,0,"[7669, 28, 26958, 40629, 70521, 2027, 46976, 1...","{0: [34, 236, 261, 357, 497, 500, 509, 1674, 2...",NaN,NaN,...,**Sense and Sensibility (1995)** is a beautifu...,**Sense and Sensibility (1995)** is a beautifu...,**Sense and Sensibility (1995)** is a beautifu...,**Sense and Sensibility (1995)** is a beautifu...,**Sense and Sensibility (1995)** is a beautifu...,**Sense and Sensibility (1995)** is a beautifu...,**Sense and Sensibility (1995)** is a beautifu...,**Sense and Sensibility (1995)** is a beautifu...,**Sense and Sensibility (1995)** is a beautifu...,**Sense and Sensibility (1995)** is a beautifu...
4,3,Grumpier Old Men (1995),Comedy|Romance,"['moldy', 'old']",**Grumpier Old Men (1995)** is a delightful se...,0,"[3450, 1179, 184349, 1929, 100083, 3258, 1473,...","{0: [357, 410, 1278, 1721, 2003, 2324, 2407, 4...",NaN,NaN,...,**Grumpier Old Men (1995)** is a delightful se...,**Grumpier Old Men (1995)** is a delightful se...,**Grumpier Old Men (1995)** is a delightful se...,**Grumpier Old Men (1995)** is a delightful se...,**Grumpier Old Men (1995)** is a delightful se...,**Grumpier Old Men (1995)** is a delightful se...,**Grump

Attacked Item IDS
[    27     74    116    446    639    731    828    918   1066   1140
   1605   1629   1730   1998   2482   2747   2851   2896   3296   4228
   4863   5804   5841   5988   6238   6731   7255   7888   7899   8604
   8605  36535  43936  65350  74685 162600 174053 190215]


In [6]:
!pip uninstall -y cornac
!pip install --quiet cornac==1.15.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 13.9 MB/s eta 0:00:00


In [7]:
import os
import zipfile
import urllib.request

import numpy as np
import pandas as pd

import cornac
from cornac.data import TextModality, FeatureModality
from cornac.eval_methods import RatioSplit
from cornac.metrics import MAE, RMSE, Recall, NDCG

# Just to verify Cornac version:
print("Cornac version:", cornac.__version__)


Cornac version: 1.15.4


In [8]:
"""

########################################################
# EXECUTION SCRIPT: Data Prep + Cornac Training (Fixed)
########################################################

import pandas as pd
from cornac.data import TextModality
from cornac.eval_methods import RatioSplit
from cornac.models import MF, ConvMF, CTR, BPR, VAECF
from cornac.metrics import RMSE, MAE, Recall, NDCG
from cornac import Experiment

top_k =10

# --------------------------
# (A) Assume df_ratings & df_items are already in memory
#     DO NOT CREATE NEW RATING DATA
# --------------------------
# Your data frames:
#   df_ratings: columns [userId, itemId, rating]
#   df_items:   columns [itemId, enriched_description]
# We do NOT redefine them here; we just transform them.

#############################
# 1) Rename columns
#############################
df_ratings_final = df_ratings.copy()
df_ratings_final.rename(columns={'userId':'user','itemId':'item'}, inplace=True)

df_items_final = df_items[['itemId','enriched_description']].copy()
df_items_final.rename(columns={'itemId':'item','enriched_description':'description'}, inplace=True)

#############################
# 2) Align Items with Ratings
#############################
# - Keep only items that appear in df_ratings_final
df_items_final.dropna(subset=['description'], inplace=True)
df_items_final.drop_duplicates(subset=['item'], keep='first', inplace=True)

# Filter out items from df_items_final that aren't in ratings
unique_items_in_ratings = df_ratings_final['item'].unique()
df_items_final = df_items_final[df_items_final['item'].isin(unique_items_in_ratings)]

# Also filter out from ratings any items that aren't in df_items_final
df_ratings_final = df_ratings_final[df_ratings_final['item'].isin(df_items_final['item'])]

# Sanity check: For exclude_unknowns=True to succeed,
# the unique item count must match in rating data vs. item side-info.
num_rating_items = df_ratings_final['item'].nunique()
num_side_items   = df_items_final['item'].nunique()
print("Items in rating data:", num_rating_items)
print("Items in side info:  ", num_side_items)

#############################
# 3) Convert to List for Cornac
#############################
feedback = df_ratings_final[['user','item','rating']].values.tolist()

item_ids = df_items_final['item'].tolist()
descriptions = df_items_final['description'].tolist()

#############################
# 4) Text Modality + RatioSplit
#############################
item_text_mod = TextModality(
    corpus=descriptions,
    ids=item_ids,
    tokenizer=None,
    max_vocab=5000
)

eval_method = RatioSplit(
    data=feedback,
    test_size=0.2,
    val_size=0.0,
    item_text=item_text_mod,
    rating_threshold=3.5,
    exclude_unknowns=True,  # crucial that items match side info
    verbose=True,
    seed=42
)

#############################
# 5) Define Your Models
#############################
# Define prototype parameters for faster runs
fast_iter = 20      # Use fewer iterations
fast_epochs = 5    # Use fewer epochs for training

# Modify models with reduced iterations/epochs
mf = MF(name='MF-Baseline', k=10, max_iter=fast_iter, learning_rate=0.01, seed=42)
convmf = ConvMF(name='ConvMF', n_epochs=fast_epochs, cnn_epochs=fast_epochs, emb_dim=10,
                lambda_u=0.01, lambda_v=0.01, seed=42)
ctr = CTR(name='CTR', k=10, max_iter=fast_iter, seed=42)
bpr = BPR(name='BPR', k=10, max_iter=fast_iter, learning_rate=0.01, lambda_reg=0.001, seed=42)
vaecf = VAECF(name='VAECF', k=10, autoencoder_structure=[32], act_fn='tanh',
              n_epochs=fast_epochs, seed=42)

# Add these to your model definitions (after line defining CTR)
cdl = CTR(name='CDL', k=20, max_iter=fast_iter, lambda_u=0.1, lambda_v=0.1, seed=42)
hft = CTR(name='HFT', k=15, max_iter=fast_iter, lambda_u=0.2, lambda_v=0.2, seed=42)

# Update models list to:
models = [mf, convmf, ctr, cdl, hft, bpr, vaecf]
#models = [mf, convmf, ctr, bpr, vaecf]

metrics = [RMSE(), MAE(), Recall(k=top_k), NDCG(k=top_k)]

#############################
# 6) Run Cornac Experiment
#############################
exp = Experiment(
    eval_method=eval_method,
    models=models,
    metrics=metrics,
    user_based=True,
    verbose=True
)
exp_result = exp.run()

# If everything is aligned, you'll no longer see AssertionError.
# The "TEST:" line will show your rating predictions & ranking metrics.
# 1) Train each model on the training set
for m in models:
    m.fit(eval_method.train_set)
"""

'\n\n########################################################\n# EXECUTION SCRIPT: Data Prep + Cornac Training (Fixed)\n########################################################\n\nimport pandas as pd\nfrom cornac.data import TextModality\nfrom cornac.eval_methods import RatioSplit\nfrom cornac.models import MF, ConvMF, CTR, BPR, VAECF\nfrom cornac.metrics import RMSE, MAE, Recall, NDCG\nfrom cornac import Experiment\n\ntop_k =10\n\n# --------------------------\n# (A) Assume df_ratings & df_items are already in memory\n#     DO NOT CREATE NEW RATING DATA\n# --------------------------\n# Your data frames:\n#   df_ratings: columns [userId, itemId, rating]\n#   df_items:   columns [itemId, enriched_description]\n# We do NOT redefine them here; we just transform them.\n\n#############################\n# 1) Rename columns\n#############################\ndf_ratings_final = df_ratings.copy()\ndf_ratings_final.rename(columns={\'userId\':\'user\',\'itemId\':\'item\'}, inplace=True)\n\ndf_items_f

In [9]:
########################################################
# EXECUTION SCRIPT: Data Prep + Cornac Training (Fixed)
########################################################

import pandas as pd
from cornac.data import TextModality
from cornac.eval_methods import RatioSplit
from cornac.models import MF, ConvMF, CTR, BPR, VAECF
from cornac.metrics import RMSE, MAE, Recall, NDCG
from cornac import Experiment

top_k =10

# --------------------------
# (A) Assume df_ratings & df_items are already in memory
#     DO NOT CREATE NEW RATING DATA
# --------------------------
# Your data frames:
#   df_ratings: columns [userId, itemId, rating]
#   df_items:   columns [itemId, enriched_description]
# We do NOT redefine them here; we just transform them.

#############################
# 1) Rename columns
#############################
df_ratings_final = df_ratings.copy()
df_ratings_final = df_ratings_final.sample(frac=0.2)
df_ratings_final.rename(columns={'userId':'user','itemId':'item'}, inplace=True)

df_items_final = df_items[['itemId','enriched_description']].copy()
df_items_final.rename(columns={'itemId':'item','enriched_description':'description'}, inplace=True)

#############################
# 2) Align Items with Ratings
#############################
# - Keep only items that appear in df_ratings_final
df_items_final.dropna(subset=['description'], inplace=True)
df_items_final.drop_duplicates(subset=['item'], keep='first', inplace=True)

# Filter out items from df_items_final that aren't in ratings
unique_items_in_ratings = df_ratings_final['item'].unique()
df_items_final = df_items_final[df_items_final['item'].isin(unique_items_in_ratings)]

# Also filter out from ratings any items that aren't in df_items_final
df_ratings_final = df_ratings_final[df_ratings_final['item'].isin(df_items_final['item'])]

# Sanity check: For exclude_unknowns=True to succeed,
# the unique item count must match in rating data vs. item side-info.
num_rating_items = df_ratings_final['item'].nunique()
num_side_items   = df_items_final['item'].nunique()
print("Items in rating data:", num_rating_items)
print("Items in side info:  ", num_side_items)

#############################
# 3) Convert to List for Cornac
#############################
feedback = df_ratings_final[['user','item','rating']].values.tolist()

item_ids = df_items_final['item'].tolist()
descriptions = df_items_final['description'].tolist()

#############################
# 4) Text Modality + RatioSplit
#############################
item_text_mod = TextModality(
    corpus=descriptions,
    ids=item_ids,
    tokenizer=None,
    max_vocab=5000
)

eval_method = RatioSplit(
    data=feedback,
    test_size=0.2,
    val_size=0.0,
    item_text=item_text_mod,
    rating_threshold=3.5,
    exclude_unknowns=True,  # crucial that items match side info
    verbose=True,
    seed=42
)

#############################################
# Define Models with Updated Training Parameters
#############################################

# Set robust training parameters
latent_dim = 50   # Increased embedding size
max_iter_mf = 100       # Number of iterations for MF-based models
n_epochs_cnn = 2        # CNN epochs (for ConvMF)
n_epochs_cf = 50        # CF epochs
learning_rate_mf = 0.005  # Learning rate for MF
lambda_reg_mf = 0.02      # Regularization for MF

# Matrix Factorization (MF) Baseline
mf = MF(
    name='MF-Baseline',
    k=latent_dim,
    max_iter=max_iter_mf,
    learning_rate=learning_rate_mf,
    lambda_reg=lambda_reg_mf,
    seed=42
)

# Convolutional Matrix Factorization (ConvMF)
convmf = ConvMF(
    name='ConvMF',
    n_epochs=n_epochs_cf,    # CF training epochs
    cnn_epochs=n_epochs_cnn, # CNN training epochs
    emb_dim=latent_dim,
    lambda_u=0.01,
    lambda_v=0.01,
    seed=42
)

# Collaborative Topic Regression (CTR)
ctr = CTR(
    name='CTR',
    k=latent_dim,
    max_iter=max_iter_mf,
    lambda_u=0.2,
    lambda_v=0.2,
    seed=42
)

# Bayesian Personalized Ranking (BPR)
bpr = BPR(
    name='BPR',
    k=latent_dim,
    max_iter=200,          # More iterations for ranking models
    learning_rate=0.01,
    lambda_reg=0.001,
    seed=42
)

# Variational Autoencoder for Collaborative Filtering (VAECF)
vaecf = VAECF(
    name='VAECF',
    k=latent_dim,
    autoencoder_structure=[64, 32],
    act_fn='tanh',
    n_epochs=n_epochs_cf,
    batch_size=256,
    seed=42
)

# Additional Models (CDL and HFT) using CTR as a baseline for side-information methods
cdl = CTR(
    name='CDL',
    k=latent_dim,
    max_iter=max_iter_mf,
    lambda_u=0.1,
    lambda_v=0.1,
    seed=42
)

hft = CTR(
    name='HFT',
    k=latent_dim,
    max_iter=max_iter_mf,
    lambda_u=0.2,
    lambda_v=0.2,
    seed=42
)

# Pack all models into a list
models = [mf, convmf, ctr, cdl, hft, bpr, vaecf]

print("Defined models with updated training parameters:")
for model in models:
    print(f" - {model.name}")


Items in rating data: 1454
Items in side info:   1454
rating_threshold = 3.5
exclude_unknowns = True
---
Training data:
Number of users = 601
Number of items = 1402
Number of ratings = 10124
Max rating = 5.0
Min rating = 0.5
Global mean = 3.6
---
Test data:
Number of users = 500
Number of items = 944
Number of ratings = 2466
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 601
Total items = 1402
Defined models with updated training parameters:
 - MF-Baseline
 - ConvMF
 - CTR
 - CDL
 - HFT
 - BPR
 - VAECF


In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
from math import log2

############################################################################
# 1. Standard & Beyond-Accuracy Metrics (Existing)
############################################################################

def compute_recall(ground_truth, recommendations):
    if not ground_truth:
        return 0.0
    hits = len(set(ground_truth) & set(recommendations))
    return hits / len(ground_truth)

def compute_precision(ground_truth, recommendations):
    if not recommendations:
        return 0.0
    hits = len(set(ground_truth) & set(recommendations))
    return hits / len(recommendations)

def compute_ndcg(ground_truth, recommendations, k):
    """Compute nDCG at k."""
    def dcg(rel_scores):
        return sum(rel / log2(pos + 2) for pos, rel in enumerate(rel_scores))
    relevance = [1 if item in ground_truth else 0 for item in recommendations[:k]]
    ideal_relevance = sorted([1] * len(ground_truth) + [0] * (k - len(ground_truth)), reverse=True)[:k]
    actual_dcg = dcg(relevance)
    ideal_dcg = dcg(ideal_relevance)
    return actual_dcg / ideal_dcg if ideal_dcg > 0 else 0.0

def compute_map(ground_truth, recommendations, k):
    """Compute Mean Average Precision at k."""
    precision_at_k = []
    hits = 0
    for i, item in enumerate(recommendations[:k]):
        if item in ground_truth:
            hits += 1
            precision_at_k.append(hits / (i + 1))
    return np.mean(precision_at_k) if precision_at_k else 0.0

def compute_hit(ground_truth, recommendations, k):
    """Returns 1 if at least one relevant item is in top-k, else 0."""
    return 1.0 if len(set(ground_truth) & set(recommendations[:k])) > 0 else 0.0


############################################################################
# 2. New Metric: MRR (Mean Reciprocal Rank)
############################################################################

def compute_mrr(ground_truth, recommendations, k):
    """
    Mean Reciprocal Rank at k:
      - If multiple relevant items, only the earliest relevant item matters.
      - If none is present in the top-k, MRR = 0 for that user.
    """
    for rank, item_id in enumerate(recommendations[:k], start=1):
        if item_id in ground_truth:
            return 1.0 / rank
    return 0.0


############################################################################
# 3. Popularity, Long-tail, Novelty, Diversity, Attack (Existing)
############################################################################

def compute_popularity_bias(recommendations, df_items):
    """
    Fraction of recommended items that are 'popular' => df_items.pop_class == 1
    """
    if not recommendations:
        return 0.0
    sub_df = df_items.loc[df_items["itemId"].isin(recommendations)]
    # If pop_class is numeric {0,1}, the mean is fraction that are '1'.
    return sub_df["pop_class"].mean()

def compute_long_tail_coverage(recommendations, df_items):
    """
    Fraction of recommended items that are 'long-tail' => df_items.pop_class == 0
    """
    if not recommendations:
        return 0.0
    sub_df = df_items.loc[df_items["itemId"].isin(recommendations)]
    num_tail = (sub_df["pop_class"] == 0).sum()
    return num_tail / len(sub_df) if len(sub_df) > 0 else 0.0

def compute_novelty(recommendations, df_items, popularity_col="popularity"):
    """
    Example novelty measure: average of 1 / log2(popularity_i + 2).
    """
    if not recommendations:
        return 0.0
    sub_df = df_items.loc[df_items["itemId"].isin(recommendations)]
    if sub_df.empty:
        return 0.0
    nov_list = []
    for _, row in sub_df.iterrows():
        pop_val = row[popularity_col] if popularity_col in row else 1.0
        nov_list.append(1.0 / np.log2(pop_val + 2))
    return np.mean(nov_list) if len(nov_list) > 0 else 0.0

def compute_intra_list_diversity(recommendations, df_items, similarity_lookup=None):
    """
    Average pairwise (1 - similarity). If no real similarity data, returns 0.
    """
    if not recommendations or len(recommendations) < 2:
        return 0.0
    pairs = 0
    total_dissim = 0.0
    for i in range(len(recommendations)):
        for j in range(i + 1, len(recommendations)):
            iid1 = recommendations[i]
            iid2 = recommendations[j]
            sim_val = 0.0
            if similarity_lookup is not None:
                sim_val = similarity_lookup.get((iid1, iid2), 0.0)
            total_dissim += (1 - sim_val)
            pairs += 1
    return total_dissim / pairs if pairs > 0 else 0.0

def compute_attack_metrics(recommendations, attacked_item_ids_list):
    """
    Coverage_indicator => 1 if any attacked item is recommended
    avg_rank => average rank of attacked items found in the top-K
    """
    if isinstance(recommendations, np.ndarray):
        recommendations = recommendations.tolist()
    if len(recommendations) == 0 or len(attacked_item_ids_list) == 0:
        return 0, 0.0
    found_positions = []
    for rank_idx, item_id in enumerate(recommendations):
        if item_id in attacked_item_ids_list:
            found_positions.append(rank_idx + 1)  # 1-based rank
    if len(found_positions) == 0:
        return 0, 0.0
    else:
        return 1, np.mean(found_positions)


############################################################################
# 4. Fairness Metric #1: Item Distribution Gini
############################################################################

def compute_item_distribution_gini(rec_df):
    """
    Compute the Gini index over item recommendation counts across all users.
    Higher Gini => more concentrated (less fair distribution).

    Steps:
      1) Flatten the recommended items from rec_df into a single list.
      2) Count how many times each item appears in top-K across all users.
      3) Compute Gini on that distribution.
    """
    # Flatten all recommended lists
    all_recs = []
    for _, row in rec_df.iterrows():
        recs = eval(row['recommendations'])
        all_recs.extend(recs)

    if not all_recs:
        return 0.0

    # Count occurrences
    item_counts = pd.Series(all_recs).value_counts().values
    # Gini calculation
    # Standard formula: G = (1 / n) * sum( (2*i - n - 1)*x_i ) / sum(x_i ) for sorted x_i
    # We'll do a simpler numeric approach:
    sorted_counts = np.sort(item_counts)  # ascending
    n = len(sorted_counts)
    cum_sum = 0
    total = sorted_counts.sum()
    for i, val in enumerate(sorted_counts, start=1):
        cum_sum += i * val
    gini = (2 * cum_sum) / (n * total) - (n + 1) / n
    return gini


############################################################################
# 5. Fairness Metric #2: User-Side Group Fairness (Active vs. Less Active)
############################################################################

def assign_user_groups(df_ratings):
    """
    Split users into two groups: 'active' (>= median # interactions)
    vs. 'less_active' (< median # interactions).
    Returns dict { user_id -> "active" or "less_active" }.
    """
    user_counts = df_ratings['userId'].value_counts()
    median_count = user_counts.median()
    group_map = {}
    for user_id, count in user_counts.items():
        if count >= median_count:
            group_map[user_id] = "active"
        else:
            group_map[user_id] = "less_active"
    return group_map

def compute_user_group_fairness(metrics_df, df_ratings):
    """
    For each user in metrics_df, find their group -> compute group-average of
    recall and ndcg. Then measure the gap.
    """
    # 1) Build user->group map
    group_map = assign_user_groups(df_ratings)

    # 2) Add group label to metrics_df
    metrics_df['user_group'] = metrics_df['user_id'].map(group_map)

    # 3) Compute group-level means
    group_stats = metrics_df.groupby('user_group')[['recall', 'ndcg']].mean().reset_index()
    # It's possible that you only have two groups, but let's do a safer approach:
    pivot_stats = group_stats.set_index('user_group').to_dict()
    recall_active = pivot_stats['recall'].get('active', 0.0)
    recall_less_active = pivot_stats['recall'].get('less_active', 0.0)
    ndcg_active = pivot_stats['ndcg'].get('active', 0.0)
    ndcg_less_active = pivot_stats['ndcg'].get('less_active', 0.0)

    # Gap = absolute difference
    recall_gap = abs(recall_active - recall_less_active)
    ndcg_gap = abs(ndcg_active - ndcg_less_active)

    return (recall_active, recall_less_active, ndcg_active, ndcg_less_active, recall_gap, ndcg_gap)


############################################################################
# 6. Semantic Drift / Text Consistency
############################################################################

def jaccard_similarity(set_a, set_b):
    intersect = len(set_a.intersection(set_b))
    union = len(set_a.union(set_b))
    return intersect / union if union > 0 else 0.0

def compute_semantic_drift(recommendations, df_items,
                           col_orig="original_description",
                           col_new="description"):
    """
    For each recommended item, we measure textual drift between the 'original_description'
    and the current 'description' using 1 - Jaccard similarity of word sets.
    We then average over the recommended items. If you have no original text, adapt accordingly.
    """
    if not recommendations:
        return 0.0

    # Filter down to recommended items
    sub_df = df_items.loc[df_items["itemId"].isin(recommendations)]
    if sub_df.empty:
        return 0.0

    drift_values = []
    for _, row in sub_df.iterrows():
        if col_orig not in row or pd.isna(row[col_orig]):
            # If no original text, skip or treat as drift=0
            continue
        if col_new not in row or pd.isna(row[col_new]):
            # If no new text, skip or treat as drift=1
            continue

        # Tokenize
        orig_tokens = set(str(row[col_orig]).lower().split())
        new_tokens = set(str(row[col_new]).lower().split())
        jacc = jaccard_similarity(orig_tokens, new_tokens)
        drift = 1.0 - jacc  # 1 - similarity
        drift_values.append(drift)

    if drift_values:
        return np.mean(drift_values)
    else:
        return 0.0


############################################################################
# 7. Compute All Metrics for a Given Model
############################################################################

def compute_model_metrics(model_name, k=10, df_items=None, attacked_item_ids_list=None,
                          similarity_lookup=None, df_ratings=None):
    """
    Compute standard ranking metrics, plus fairness, plus semantic drift.

    NEW parameters:
      - df_ratings: required for user-group fairness
      - df_items:   must contain both 'description' and 'original_description' columns
                   (for semantic drift), as well as 'popularity' or 'pop_class' as needed.
    """
    # Load the CSV of user recommendations
    rec_df = pd.read_csv(f'recs_{model_name}.csv')

    # We'll store per-user metrics for subsequent user-group fairness computations
    user_metrics = []

    for _, row in rec_df.iterrows():
        user_id = row['user_id']
        gt = eval(row['ground_truth'])        # ground-truth relevant items
        recs = eval(row['recommendations'])   # top-k recommended items

        # 1) Standard metrics
        recall_val = compute_recall(gt, recs)
        precision_val = compute_precision(gt, recs)
        ndcg_val = compute_ndcg(gt, recs, k)
        map_val = compute_map(gt, recs, k)
        hit_val = compute_hit(gt, recs, k)
        mrr_val = compute_mrr(gt, recs, k)

        # 2) Pop/LongTail
        pop_bias_val = compute_popularity_bias(recs, df_items)
        long_tail_val = compute_long_tail_coverage(recs, df_items)

        # 3) Novelty & Diversity
        novelty_val = compute_novelty(recs, df_items, popularity_col="popularity")
        ild_val = compute_intra_list_diversity(recs, df_items, similarity_lookup=similarity_lookup)

        # 4) Attack metrics
        coverage_indicator, avg_rank_val = compute_attack_metrics(recs, attacked_item_ids_list)

        # 5) Semantic Drift
        drift_val = compute_semantic_drift(recs, df_items,
                                           col_orig="original_description",
                                           col_new="description")

        user_metrics.append({
            'user_id': user_id,
            'recall': recall_val,
            'precision': precision_val,
            'ndcg': ndcg_val,
            'mean_avg_precision': map_val,
            'hit': hit_val,
            'mrr': mrr_val,
            'pop_bias': pop_bias_val,
            'long_tail': long_tail_val,
            'novelty': novelty_val,
            'diversity': ild_val,
            'attack_cov': coverage_indicator,
            'attack_rank': avg_rank_val,
            'semantic_drift': drift_val
        })

    metrics_df = pd.DataFrame(user_metrics)
    metrics_df.to_csv(f'metrics_{model_name}.csv', index=False)

    # 6) Summaries
    # Attack coverage
    attack_coverage = metrics_df['attack_cov'].mean()
    filtered_attack_ranks = metrics_df.loc[metrics_df['attack_cov'] == 1, 'attack_rank']
    avg_attack_rank = filtered_attack_ranks.mean() if len(filtered_attack_ranks) else 0.0

    # Item distribution Gini
    # We compute Gini from the same "recs_{model_name}.csv"
    item_gini = compute_item_distribution_gini(rec_df)

    # User-side group fairness
    # This yields recall_active, recall_less_active, ndcg_active, ndcg_less_active, recall_gap, ndcg_gap
    if df_ratings is not None:
        (ra, rl, na, nl, rg, ng) = compute_user_group_fairness(metrics_df, df_ratings)
    else:
        ra = rl = na = nl = rg = ng = 0.0

    return {
        'Model': model_name,
        # standard
        'Recall': metrics_df['recall'].mean(),
        'Precision': metrics_df['precision'].mean(),
        'NDCG': metrics_df['ndcg'].mean(),
        'MAP': metrics_df['mean_avg_precision'].mean(),
        'Hit@K': metrics_df['hit'].mean(),
        'MRR': metrics_df['mrr'].mean(),
        # pop/longtail
        'PopBias': metrics_df['pop_bias'].mean(),
        'LongTail': metrics_df['long_tail'].mean(),
        # new
        'Novelty': metrics_df['novelty'].mean(),
        'Diversity': metrics_df['diversity'].mean(),
        # attack
        'AttackCoverage': attack_coverage,
        'AvgAttackRank': avg_attack_rank,
        # semantic drift
        'SemanticDrift': metrics_df['semantic_drift'].mean(),
        # fairness
        'ItemDistGini': item_gini,
        'Recall_Active': ra,
        'Recall_LessActive': rl,
        'NDCG_Active': na,
        'NDCG_LessActive': nl,
        'Recall_Gap': rg,
        'NDCG_Gap': ng
    }


############################################################################
# 8. Generate Recommendations & Execute Full Pipeline
############################################################################

def generate_model_recommendations(model, train_set, test_set, k=10, remove_seen=True):
    """Generate and save recommendations for a single model."""
    reverse_uid_map = {v: k for k, v in train_set.uid_map.items()}
    reverse_iid_map = {v: k for k, v in train_set.iid_map.items()}

    recommendations = {}
    test_users, test_items, test_ratings = test_set.uir_tuple

    user_to_test_items = defaultdict(list)
    for u_idx, i_idx, rating in zip(test_users, test_items, test_ratings):
        if i_idx in reverse_iid_map:
            user_to_test_items[u_idx].append((i_idx, rating))

    for user_idx in test_set.user_indices:
        user_id = reverse_uid_map[user_idx]
        gt_items = [
            reverse_iid_map[i_idx]
            for i_idx, rating in user_to_test_items.get(user_idx, [])
            if rating >= 3.5
        ]

        all_items = list(reverse_iid_map.keys())
        scores = np.array([model.score(user_idx, i_idx) for i_idx in all_items])

        if remove_seen:
            seen_mask = train_set.uir_tuple[0] == user_idx
            seen_items = train_set.uir_tuple[1][seen_mask]
            valid_mask = ~np.isin(all_items, seen_items)
            scores = scores[valid_mask]
            valid_items = np.array(all_items)[valid_mask]
        else:
            valid_items = np.array(all_items)

        top_k_idx = np.argsort(scores)[-k:][::-1]
        recs = [reverse_iid_map[valid_items[idx]] for idx in top_k_idx]
        rec_scores = scores[top_k_idx].tolist()

        recommendations[user_id] = {
            'user_id': user_id,
            'ground_truth': gt_items,
            'recommendations': recs,
            'scores': rec_scores
        }

    rec_df = pd.DataFrame.from_dict(recommendations, orient='index')
    rec_df.to_csv(f'recs_{model.name}.csv', index=False)
    return rec_df


def execute_full_pipeline(models, eval_method, k=10, df_items=None,
                          attacked_item_ids_list=None, similarity_lookup=None,
                          df_ratings=None):
    """
    End-to-end execution with new metrics.
      - df_ratings is used for user-group fairness.
      - df_items is used for side-info metrics like popularity, text columns, etc.
    """
    print("=== Recommendation Generation Phase ===")
    for model in models:
        print(f"Generating recommendations for {model.name}...")
        generate_model_recommendations(model, eval_method.train_set, eval_method.test_set, k)

    print("\n=== Evaluation Phase ===")
    all_metrics = []
    for model in models:
        print(f"Evaluating {model.name}...")
        metrics_res = compute_model_metrics(
            model.name,
            k=k,
            df_items=df_items,
            attacked_item_ids_list=attacked_item_ids_list,
            similarity_lookup=similarity_lookup,
            df_ratings=df_ratings  # needed for user-group fairness
        )
        all_metrics.append(metrics_res)

    final_df = pd.DataFrame(all_metrics)
    print("\n=== Final Results ===")
    display(final_df)
    return final_df


############################################################################
# EXAMPLE USAGE:
############################################################################
similarity_lookup = {}
# 1) FIT each model
for m in models:
    print(f"Fitting {m.name} ...")
    m.fit(eval_method.train_set)

# 2) THEN generate recommendations & evaluate
results = execute_full_pipeline(
    models,
    eval_method,
    k=10,
    df_items=df_items,
    attacked_item_ids_list=attacked_item_ids_list,
    similarity_lookup=similarity_lookup,
    df_ratings=df_ratings
)


Fitting MF-Baseline ...
Fitting ConvMF ...
Fitting CTR ...


  0%|          | 0/100 [00:00<?, ?it/s]

Learning completed!
Fitting CDL ...


  0%|          | 0/100 [00:00<?, ?it/s]

Learning completed!
Fitting HFT ...


  0%|          | 0/100 [00:00<?, ?it/s]

Learning completed!
Fitting BPR ...
Fitting VAECF ...
=== Recommendation Generation Phase ===
Generating recommendations for MF-Baseline...
Generating recommendations for ConvMF...
Generating recommendations for CTR...
Generating recommendations for CDL...
Generating recommendations for HFT...
Generating recommendations for BPR...
Generating recommendations for VAECF...

=== Evaluation Phase ===
Evaluating MF-Baseline...
Evaluating ConvMF...
Evaluating CTR...
Evaluating CDL...
Evaluating HFT...
Evaluating BPR...
Evaluating VAECF...

=== Final Results ===


,Model,Recall,Precision,NDCG,MAP,Hit@K,MRR,PopBias,LongTail,Novelty,...,AttackCoverage,AvgAttackRank,SemanticDrift,ItemDistGini,Recall_Active,Recall_LessActive,NDCG_Active,NDCG_LessActive,Recall_Gap,NDCG_Gap
0,MF-Baseline,0.009952,0.004501,0.005873,0.008269,0.043053,0.008174,0.566341,0.518787,0.63093,...,0.000000,0.000000,0.0,0.697109,0.013663,0.004717,0.008982,0.001488,0.008946,0.007494
1,ConvMF,0.002957,0.001370,0.001835,0.003082,0.013699,0.003082,0.897260,0.348728,0.63093,...,0.003914,9.000000,0.0,0.875816,0.001709,0.004717,0.001945,0.001680,0.003008,0.000265
2,CTR,0.039618,0.011742,0.024853,0.034534,0.111546,0.035773,0.126419,0.889628,0.63093,...,0.021526,5.090909,0.0,0.538067,0.030194,0.052909,0.020537,0.030941,0.022715,0.010404
3,CDL,0.039956,0.011546,0.025303,0.036526,0.107632,0.036958,0.133659,0.884540,0.63093,...,0.025440,5.230769,0.0,0.535299,0.029491,0.054717,0.020309,0.032347,0.025226,0.012038
4,HFT,0.039618,0.011742,0.024853,0.034534,0.111546,0.035773,0.126419,0.889628,0.63093,...,0.021526,5.090909,0.0,0.538067,0.030194,0.052909,0.020537,0.030941,0.022715,0.010404
5,BPR,0.056274,0.015851,0.034592,0.045972,0.150685,0.046559,0.003523,0.996477,0.63093,...,0.000000,0.000000,0.0,0.831498,0.045337,0.071698,0.032282,0.037850,0.026361,0.005568
6,VAECF,0.058739,0.017025,0.039597,0.055464,0.162427,0.055364,0.000000,1.000000,0.63093,...,0.000000,0.000000,0.0,0.417551,0.052059,0.068160,0.038334,0.041378,0.016101,0.003044


In [ ]:
results.columns

Index(['Model', 'Recall', 'Precision', 'NDCG', 'MAP', 'Hit@K', 'MRR',
       'PopBias', 'LongTail', 'Novelty', 'Diversity', 'AttackCoverage',
       'AvgAttackRank', 'SemanticDrift', 'ItemDistGini', 'Recall_Active',
       'Recall_LessActive', 'NDCG_Active', 'NDCG_LessActive', 'Recall_Gap',
       'NDCG_Gap'],
      dtype='object')

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
# -------------
# Metric computation functions
# -------------

def compute_recall(ground_truth, recommendations):
    if not ground_truth:
        return 0.0
    hits = len(set(ground_truth) & set(recommendations))
    return hits / len(ground_truth)

def compute_precision(ground_truth, recommendations):
    if not recommendations:
        return 0.0
    hits = len(set(ground_truth) & set(recommendations))
    return hits / len(recommendations)

def compute_ndcg(ground_truth, recommendations, k):
    import numpy as np
    def dcg(relevant_scores):
        return sum(score / np.log2(pos + 2) for pos, score in enumerate(relevant_scores))

    relevance = [1 if item in ground_truth else 0 for item in recommendations[:k]]
    ideal_relevance = sorted([1]*len(ground_truth) + [0]*(k-len(ground_truth)), reverse=True)[:k]

    return dcg(relevance) / dcg(ideal_relevance) if sum(ideal_relevance) > 0 else 0.0

def compute_map(ground_truth, recommendations, k):
    import numpy as np
    precision_at_k = []
    hits = 0
    for i, item in enumerate(recommendations[:k]):
        if item in ground_truth:
            hits += 1
            precision_at_k.append(hits / (i + 1))
    return np.mean(precision_at_k) if precision_at_k else 0.0

def compute_hit(ground_truth, recommendations, k):
    return 1.0 if len(set(ground_truth) & set(recommendations[:k])) > 0 else 0.0

# ----------------
# NEW METRICS: Popularity & Long Tail
# ----------------
def compute_popularity_bias(recommendations, df_items):
    """
    Returns the fraction of recommended items that are in the 'popular' class
    pop_class == 1 => popular
    """
    if not recommendations:
        return 0.0
    sub_df = df_items.loc[df_items["itemId"].isin(recommendations)]
    # If pop_class is numeric {0,1}, the average is fraction that are '1'.
    return sub_df["pop_class"].mean()

def compute_long_tail_coverage(recommendations, df_items):
    """
    Returns the fraction of recommended items that are in the 'long tail'.
    pop_class == 0 => tail
    """
    if not recommendations:
        return 0.0
    sub_df = df_items.loc[df_items["itemId"].isin(recommendations)]
    # fraction that are '0'
    num_tail = (sub_df["pop_class"] == 0).sum()
    return num_tail / len(sub_df)

#######################
# Additional Metrics
#######################

def compute_novelty(recommendations, df_items, popularity_col="popularity"):
    """
    Example novelty measure: average of 1 / log2(popularity_i + 2).
    Make sure df_items[popularity_col] exists for each recommended item.
    """
    if not recommendations:
        return 0.0
    sub_df = df_items.loc[df_items["itemId"].isin(recommendations)]
    if sub_df.empty:
        return 0.0
    # If popularity is 0-based, add 1 or 2 to avoid log(0)
    nov_list = []
    for _, row in sub_df.iterrows():
        pop_val = row[popularity_col] if popularity_col in row else 1.0
        nov_list.append(1.0 / np.log2(pop_val + 2))
    return np.mean(nov_list)

def compute_intra_list_diversity(recommendations, df_items, similarity_lookup=None):
    """
    Average pairwise dissimilarity = 1 - similarity across the recommended set.
    'similarity_lookup' is a dict or function that gives sim(i,j) in [0,1].
    If you don't have a real similarity, this can be a placeholder.
    """
    if not recommendations or len(recommendations) < 2:
        return 0.0
    # We'll compute mean(1 - sim) over pairs
    pairs = 0
    total_dissim = 0.0
    for i in range(len(recommendations)):
        for j in range(i + 1, len(recommendations)):
            iid1 = recommendations[i]
            iid2 = recommendations[j]
            sim_val = 0.0
            if similarity_lookup is not None:
                sim_val = similarity_lookup.get((iid1, iid2), 0.0)
            # dissimilarity = 1 - sim
            total_dissim += (1 - sim_val)
            pairs += 1
    return total_dissim / pairs if pairs > 0 else 0.0

def compute_attack_metrics(recommendations, attacked_item_ids_list):
    # Convert to a standard list if you want
    if isinstance(recommendations, np.ndarray):
        recommendations = recommendations.tolist()

    # Check empties by length
    if len(recommendations) == 0 or len(attacked_item_ids_list) == 0:
        return 0, 0.0

    found_positions = []
    for rank_idx, item_id in enumerate(recommendations):
        if item_id in attacked_item_ids_list:
            found_positions.append(rank_idx + 1)

    if len(found_positions) == 0:
        return 0, 0.0
    else:
        avg_rank = np.mean(found_positions)
        return 1, avg_rank


In [ ]:
def compute_model_metrics(model_name, k=10, df_items=None,
                          attacked_item_ids_list=None,
                          similarity_lookup=None):
    """Compute metrics from saved recommendations, including new metrics (novelty, diversity, attack coverage...)."""
    # Load recommendations
    rec_df = pd.read_csv(f'recs_{model_name}.csv')

    metrics = []
    for _, row in rec_df.iterrows():
        gt = eval(row['ground_truth'])
        recs = eval(row['recommendations'])

        # 1) Standard metrics
        recall_val = compute_recall(gt, recs)
        precision_val = compute_precision(gt, recs)
        ndcg_val = compute_ndcg(gt, recs, k)
        map_val = compute_map(gt, recs, k)
        hit_val = compute_hit(gt, recs, k)

        # 2) Pop/LongTail
        pop_bias_val = compute_popularity_bias(recs, df_items)
        long_tail_val = compute_long_tail_coverage(recs, df_items)

        # 3) New: Novelty & Diversity
        novelty_val = compute_novelty(recs, df_items, popularity_col="popularity")
        ild_val = compute_intra_list_diversity(recs, df_items, similarity_lookup=similarity_lookup)

        # 4) Attack metrics
        coverage_indicator, avg_rank_val = compute_attack_metrics(recs, attacked_item_ids_list)

        metrics.append({
            'user_id': row['user_id'],
            # standard
            'recall': recall_val,
            'precision': precision_val,
            'ndcg': ndcg_val,
            'mean_avg_precision': map_val,
            'hit': hit_val,
            # pop/longtail
            'pop_bias': pop_bias_val,
            'long_tail': long_tail_val,
            # new
            'novelty': novelty_val,
            'diversity': ild_val,
            'attack_cov': coverage_indicator,  # 1 or 0 for each user
            'attack_rank': avg_rank_val        # average rank if found
        })

    metrics_df = pd.DataFrame(metrics)
    metrics_df.to_csv(f'metrics_{model_name}.csv', index=False)

    # Summaries
    attack_coverage = metrics_df['attack_cov'].mean()  # fraction of users who got attacked item
    # among only those users who got attacked item
    filtered_attack_ranks = metrics_df.loc[metrics_df['attack_cov'] == 1, 'attack_rank']
    avg_attack_rank = filtered_attack_ranks.mean() if len(filtered_attack_ranks) else 0.0

    return {
        'Model': model_name,
        'Recall': metrics_df['recall'].mean(),
        'Precision': metrics_df['precision'].mean(),
        'NDCG': metrics_df['ndcg'].mean(),
        'MAP': metrics_df['mean_avg_precision'].mean(),
        'Hit@K': metrics_df['hit'].mean(),
        'PopBias': metrics_df['pop_bias'].mean(),
        'LongTail': metrics_df['long_tail'].mean(),
        'Novelty': metrics_df['novelty'].mean(),
        'Diversity': metrics_df['diversity'].mean(),
        'AttackCoverage': attack_coverage,
        'AvgAttackRank': avg_attack_rank
    }



In [ ]:
# --------------------------
# 1. Recommendation Generation Phase
# --------------------------
def generate_model_recommendations(model, train_set, test_set, k=10, remove_seen=True):
    """Generate and save recommendations for a single model"""
    # Create reverse mappings
    reverse_uid_map = {v: k for k, v in train_set.uid_map.items()}
    reverse_iid_map = {v: k for k, v in train_set.iid_map.items()}

    recommendations = {}
    test_users, test_items, test_ratings = test_set.uir_tuple

    # Build test interactions index
    user_to_test_items = defaultdict(list)
    for u_idx, i_idx, rating in zip(test_users, test_items, test_ratings):
        if i_idx in reverse_iid_map:
            user_to_test_items[u_idx].append((i_idx, rating))

    # Generate recommendations per user
    for user_idx in test_set.user_indices:
        user_id = reverse_uid_map[user_idx]

        # Ground truth collection
        gt_items = [
            reverse_iid_map[i_idx]
            for i_idx, rating in user_to_test_items.get(user_idx, [])
            if rating >= 3.5
        ]

        # Score calculation
        all_items = list(reverse_iid_map.keys())
        scores = np.array([model.score(user_idx, i_idx) for i_idx in all_items])

        # Seen item filtering
        if remove_seen:
            seen_mask = train_set.uir_tuple[0] == user_idx
            seen_items = train_set.uir_tuple[1][seen_mask]
            valid_mask = ~np.isin(all_items, seen_items)
            scores = scores[valid_mask]
            valid_items = np.array(all_items)[valid_mask]
        else:
            valid_items = np.array(all_items)

        # Recommendation extraction
        top_k_idx = np.argsort(scores)[-k:][::-1]
        recs = [reverse_iid_map[valid_items[idx]] for idx in top_k_idx]
        rec_scores = scores[top_k_idx].tolist()

        recommendations[user_id] = {
            'user_id': user_id,
            'ground_truth': gt_items,
            'recommendations': recs,
            'scores': rec_scores
        }

    # Save to CSV
    rec_df = pd.DataFrame.from_dict(recommendations, orient='index')
    rec_df.to_csv(f'recs_{model.name}.csv', index=False)
    return rec_df

def execute_full_pipeline(models, eval_method, k=10, df_items=None,
                          attacked_item_ids_list=None, similarity_lookup=None):
    """End-to-end execution with new metrics included."""
    # Phase 1: Generate recommendations
    print("=== Recommendation Generation Phase ===")
    for model in models:
        print(f"Generating recommendations for {model.name}...")
        generate_model_recommendations(model, eval_method.train_set, eval_method.test_set, k)

    # Phase 2: Compute metrics
    print("\n=== Evaluation Phase ===")
    all_metrics = []
    for model in models:
        print(f"Evaluating {model.name}...")
        metrics_res = compute_model_metrics(
            model.name,
            k=k,
            df_items=df_items,
            attacked_item_ids_list=attacked_item_ids_list,
            similarity_lookup=similarity_lookup
        )
        all_metrics.append(metrics_res)

    final_df = pd.DataFrame(all_metrics)
    print("\n=== Final Results ===")
    display(final_df)
    return final_df

# Example usage:
similarity_lookup = {}  # or you define a real dictionary
results = execute_full_pipeline(
    models, eval_method, k=10, df_items=df_items,
    attacked_item_ids_list=attacked_item_ids_list,
    similarity_lookup=similarity_lookup
)


=== Recommendation Generation Phase ===
Generating recommendations for MF-Baseline...
Generating recommendations for ConvMF...
Generating recommendations for CTR...
Generating recommendations for CDL...
Generating recommendations for HFT...
Generating recommendations for BPR...
Generating recommendations for VAECF...

=== Evaluation Phase ===
Evaluating MF-Baseline...
Evaluating ConvMF...
Evaluating CTR...
Evaluating CDL...
Evaluating HFT...
Evaluating BPR...
Evaluating VAECF...

=== Final Results ===


,Model,Recall,Precision,NDCG,MAP,Hit@K,PopBias,LongTail,Novelty,Diversity,AttackCoverage,AvgAttackRank
0,MF-Baseline,0.012160,0.021122,0.017737,0.030583,0.156766,0.709241,0.389604,0.63093,1.0,0.00000,0.0
1,ConvMF,0.012282,0.019142,0.020034,0.042453,0.148515,0.048845,0.970297,0.63093,1.0,0.00165,8.0
2,CTR,0.142756,0.145710,0.183819,0.282177,0.645215,0.173267,0.830363,0.63093,1.0,0.00330,8.5
3,CDL,0.138060,0.134818,0.169327,0.266491,0.638614,0.139934,0.862541,0.63093,1.0,0.00330,4.0
4,HFT,0.137144,0.132838,0.172026,0.285816,0.658416,0.163861,0.838284,0.63093,1.0,0.00330,9.0
5,BPR,0.102293,0.131848,0.171945,0.298822,0.561056,0.000000,1.000000,0.63093,1.0,0.00000,0.0
6,VAECF,0.034602,0.048515,0.053175,0.107393,0.320132,0.006766,0.993234,0.63093,1.0,0.00000,0.0
